In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import xgboost as xgb 
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
df.date = df.date.values.astype('M8[ns]')
df.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df.loc[:, 'CPM'] = df.apply(lambda x: weird_division(x.total_revenue * 100, x.measurable_impressions) * 1000 , axis=1)

split_date = '2019-06-21'

df_train = df.loc[df.date <= split_date]
df_test = df.loc[df.date > split_date]

In [ ]:
def preprocess(df, percentile=95):
    df = df.loc[df.CPM >= 0]
    return df.loc[df.CPM < df.CPM.quantile(percentile / 100)]

df_train_pp = preprocess(df_train)
df_test_pp = preprocess(df_test)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                          colsample_bytree=0.3, 
                          learning_rate=0.05,
                          max_depth=7, 
                          alpha=10, 
                          n_estimators=1000)

In [ ]:
xg_reg.fit(df_train_pp.drop(['CPM', 'date', 'total_revenue'], axis=1), 
           df_train_pp.CPM)

In [ ]:
pred = xg_reg.predict(df_test_pp.drop(['CPM', 'date', 'total_revenue'], axis=1))

In [ ]:
print(f'Mean Squared Error: {mean_squared_error(df_test_pp.CPM, pred)}')